In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!wget -O /Users/toshikifukui/Desktop/Covid-19/covid-19.csv https://stopcovid19.metro.tokyo.lg.jp/data/130001_tokyo_covid19_patients.csv

--2022-03-07 20:47:58--  https://stopcovid19.metro.tokyo.lg.jp/data/130001_tokyo_covid19_patients.csv
stopcovid19.metro.tokyo.lg.jp (stopcovid19.metro.tokyo.lg.jp) をDNSに問いあわせています... 54.255.56.197, 52.220.193.16
stopcovid19.metro.tokyo.lg.jp (stopcovid19.metro.tokyo.lg.jp)|54.255.56.197|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 92912552 (89M) [text/csv]
`/Users/toshikifukui/Desktop/Covid-19/covid-19.csv' に保存中

/Users/toshikifukui 100%[===================>]  88.61M  4.18MB/s 時間 20s        

2022-03-07 20:48:19 (4.46 MB/s) - `/Users/toshikifukui/Desktop/Covid-19/covid-19.csv' へ保存完了 [92912552/92912552]



In [5]:
corona_data = pd.read_csv("/Users/toshikifukui/Desktop/Covid-19/covid-19.csv")
corona_data

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,6,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,No,全国地方公共団体コード,都道府県名,市区町村名,公表_年月日,発症_年月日,確定_年月日,患者_居住地,患者_年代,患者_性別,患者_職業,患者_状態,患者_症状,患者_渡航歴の有無フラグ,患者_接触歴の有無フラグ,備考,退院済フラグ
0,1,130001,東京都,NaN,2020-01-24,NaN,NaN,湖北省武漢市,40代,男性,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2,130001,東京都,NaN,2020-01-25,NaN,NaN,湖北省武漢市,30代,女性,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,3,130001,東京都,NaN,2020-01-30,NaN,NaN,湖南省長沙市,30代,女性,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,4,130001,東京都,NaN,2020-02-13,NaN,NaN,都内,70代,男性,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,5,130001,東京都,NaN,2020-02-14,NaN,NaN,都内,50代,女性,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100229,100184,130001,東京都,NaN,2021-02-01,NaN,NaN,NaN,30代,女性,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100230,100185,130001,東京都,NaN,2021-02-01,NaN,NaN,NaN,70代,男性,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100231,100186,130001,東京都,NaN,2021-02-01,NaN,NaN,NaN,10代,女性,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100232,100187,130001,東京都,NaN,2021-02-01,NaN,NaN,NaN,70代,女性,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
corona_new = corona_data["公表_年月日"].value_counts().sort_index()
corona_new

2020-01-24       1
2020-01-25       1
2020-01-30       1
2020-02-13       1
2020-02-14       2
              ... 
2021-01-28    1064
2021-01-29     868
2021-01-30     769
2021-01-31     633
2021-02-01     393
Name: 公表_年月日, Length: 349, dtype: int64

In [7]:
corona_df = pd.DataFrame(corona_new)
corona_df

,公表_年月日
2020-01-24,1
2020-01-25,1
2020-01-30,1
2020-02-13,1
2020-02-14,2
...,...
2021-01-28,1064
2021-01-29,868
2021-01-30,769
2021-01-31,633


In [8]:
corona_df.columns = ["１日あたりの新規感染者数"]

In [9]:
corona_df

,１日あたりの新規感染者数
2020-01-24,1
2020-01-25,1
2020-01-30,1
2020-02-13,1
2020-02-14,2
...,...
2021-01-28,1064
2021-01-29,868
2021-01-30,769
2021-01-31,633


In [10]:
#訓練データとテストデータに分割
test_size = 0.3
sp = corona_df["１日あたりの新規感染者数"].values# pandasからnumpyへ変換
total_len = len(sp)
test_len = int(len(sp)*test_size)#テストサイズ
train_len = total_len - test_len#学習サイズ
print("total_len:", total_len, " train_len:", train_len, "test_len:", test_len)

train = sp[0:train_len]
test = sp[train_len:total_len]

#min, max標準化
#メリット...正規化することで学習速度を向上させる、複数の説明変数で値の大きさがバラバラで、学習精度に影響を与えることを防ぐため
x_max, x_min = max(sp), min(sp)
train_norm = (train - x_min) / (x_max - x_min)
test_norm = (test - x_min) / (x_max - x_min)
n_time = 14#何日間のデータを参考にして予測するか
x_train,x_test,y_train,y_test = [], [], [], []
for i in range(train_len-n_time):
  x_train.append(train_norm[i:i+n_time].reshape(-1,1))#x_trainに6つのデータを追加
  y_train.append(train_norm[i+n_time])#y_trainに1つのデータを追加
for i in range(test_len-n_time):
  x_test.append(test_norm[i:i+n_time])
  y_test.append(test_norm[i+n_time])

total_len: 349  train_len: 245 test_len: 104


In [11]:
#学習データをテンソル型に変換
batch_size = 128
input_data = torch.FloatTensor(x_train)
correct_data = torch.FloatTensor(y_train)
train = TensorDataset(input_data,correct_data)
train_loader = DataLoader(train, batch_size=batch_size)

In [12]:
#LSTMのモデルを定義
class DNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = x.to(device)
        y_lstm, h = self.lstm(x, None)
        y = self.fc1(y_lstm[:, -1, :])
        return y

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = DNN(1, 200 ,1).to(device)
print(model)

cpu
DNN(
  (lstm): LSTM(1, 200, batch_first=True)
  (fc1): Linear(in_features=200, out_features=1, bias=True)
)


In [13]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)
#学習
record_train_loss = []
num_epochs = 700
for i in range(num_epochs):
  total_loss = 0
  for j, (x, t) in enumerate(train_loader):
      x, t = x.to(device), t.reshape(-1,1).to(device)
#        x, t = x.to(device), t.to(device)
      y = model(x)
      loss = criterion(y, t)
      total_loss += loss
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  total_loss /= j+1
  record_train_loss.append(loss)
  if i % 50 == 0:
    print(f'epoch: {i:3} loss: {loss.item():10.8f}')

In [1]:
#予測
#trainの部分で構成したモデルを用いることで残りの部分を予測
sp_norm = sp/x_max #終値を正規化
in_true = list(sp_norm.reshape(-1).astype(np.float32)) # ここの意味を理解する#sp_norm.reshape(-1)で1行に直す #float32(GPU)型に直す
pred = []
model.eval()
for i in range(n_time):
    pred.append(0)
    day = -3
with torch.no_grad():
    for i in range(len(sp)):
        x = torch.tensor(in_true[i:i+n_time])  # 直近の時系列を取り出す #前の6つのデータから予測
        x = x.reshape(1, -1, 1)  # (バッチサイズ, 時系列の数, 入力の数)#reshapeすることで1,6,1の行列に変換することができる
        y = model(x)
        pred.append(y[0].item())  # 予測結果をpredictedに追加する
        if i > len(sp)-9:
            if day < 0:
                print("{}日前の新規感染者数：{}人".format(abs(day), round(y[0].item()*x_max)))
            elif day == 0:
                print("今日の新規感染者数：{}人".format(round(y[0].item()*x_max)))
            else:
                print("{}日後の新規感染者数：{}人".format(day,round(y[0].item()*x_max)))
            day += 1

plt.plot(range(len(sp)), sp, label="Correct")
pred = np.array(pred)
pred = pred * x_max
plt.plot(range(len(pred)), pred, label="Predicted")
plt.ylabel("New Confirmed Cases per day")
plt.xlabel("Day")
plt.title("COVID-19")
plt.legend()
plt.show()

NameError: name 'sp' is not defined